In [10]:
#基本的引入
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
#from tensorboardX import SummaryWriter
import matplotlib.pyplot as plt
import numpy as np
import os
import cv2
import random
#为了minibatch的功能
import torch.utils.data as Data
from torchvision.transforms import transforms
from torch.utils.data import DataLoader, Dataset

In [ ]:
#因为在kaggle上没有这个功能我就没有用
from torch.utils.tensorboard import SummaryWriter       
writer = SummaryWriter(PATH_to_log_dir)

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
# 设置随机数种子
setup_seed(20)

In [ ]:
#改编版！新增monitor
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, monitor="val_acc", patience=7, verbose=False, delta=0):
        """
        Args:
            monitor (string): 可以选 "val_acc"or "val_loss"
                            
                            Default: "val_acc"
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
        """
        self.monitor=monitor
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.val_acc_max = 0
        self.delta = delta
    
    def __call__(self, val, model):
        if self.monitor=='val_loss':
            val_loss=val
            score = -val_loss

            if self.best_score is None:
                self.best_score = score
                self.save_checkpoint(val_loss, model)
            elif score < self.best_score + self.delta:
                self.counter += 1
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.save_checkpoint(val_loss, model)
                self.counter = 0
        elif self.monitor=='val_acc':
            #这里的val是0-100之间的数。
            val_acc=val
            score = val_acc
            if self.best_score is None:
                self.best_score = score
                self.save_checkpoint(val_acc, model)
            elif score < self.best_score + self.delta:
                self.counter += 1
                print(f'EarlyStopping counter: {self.counter} out of {self.patience}')
                if self.counter >= self.patience:
                    self.early_stop = True
            else:
                self.best_score = score
                self.save_checkpoint(val_acc, model)
                self.counter = 0   

    def save_checkpoint(self, val, model):
        '''Saves model when validation loss decrease.'''
        '''Saves model when validation accuracy increase.'''
        if self.monitor=='val_loss':
            if self.verbose:
                print(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val:.6f}).  Saving model ...')           
            self.val_loss_min = val
        if self.monitor=='val_acc':
            if self.verbose:
                print(f'Validation accuracy increased ({self.val_acc_max:.6f}% --> {val:.6f}%).  Saving model ...')
            self.val_acc_max = val
        torch.save(model.state_dict(), 'checkpoint.pt')	# 这里会存储迄今最优模型的参数

使用更成熟的ImageFolder可以减轻内存压力。而且少写好多代码。

In [ ]:
#正常的transform
transform1 = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
#算是一种data augmentation
transform2 = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.RandomHorizontalFlip(),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [ ]:
train_data=torchvision.datasets.ImageFolder('./Caltech101/train',transform=transform1)
eval_data=torchvision.datasets.ImageFolder('./Caltech101/eval',transform=transform1)
test_data=torchvision.datasets.ImageFolder('./Caltech101/test',transform=transform1)
Num_workers=2
train_loader=Data.DataLoader(dataset=train_data,batch_size=32,
                             shuffle=True, num_workers=Num_workers)
valid_loader=Data.DataLoader(dataset=eval_data,batch_size=32,
                             shuffle=True, num_workers=Num_workers)
test_loader=Data.DataLoader(dataset=test_data,batch_size=32,
                             shuffle=True, num_workers=Num_workers)

In [ ]:
#CNN   15MB 经济实惠，下载不用等太久
#去除drop，减少全连接层后，训练速度和之前差不多，但在后期valid acc 65%时出现overfitting，即后劲不足
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (3, 224, 224)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=8,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=1,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (8, 224, 224)
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (8, 112, 112)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(8),              #使用这个normalize后，训练速度变快了！      
        )
        self.conv2 = nn.Sequential(         # input shape (8, 112, 112)
            nn.Conv2d(8, 16, 3, 1, 1),     # output shape (16, 112, 112)
            nn.MaxPool2d(2),                # output shape (16, 56, 56)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(16),        
        )
        self.conv3 = nn.Sequential(         # input shape (16, 56, 56)
            nn.Conv2d(16,32, 3, 1, 1),     # output shape (32, 56, 56)
            nn.MaxPool2d(2),                # output shape (32, 28, 28)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(32),  
        )
        self.conv4 = nn.Sequential(         # input shape (32, 28, 28)
            nn.Conv2d(32,64, 3, 1, 1),     # output shape (64, 28, 28)
            nn.MaxPool2d(2),                # output shape (64, 14, 14)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(64),
        )
        self.conv5 = nn.Sequential(         # input shape (64, 14, 14)
            nn.Conv2d(64, 128, 3, 1, 1),     # output shape (128, 14, 14)
            nn.MaxPool2d(2),                # output shape (128, 7, 7)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(128),
        )
        self.conv6 = nn.Sequential(         # input shape (128, 7, 7)
            nn.Conv2d(128, 256, 3, 1, 1),     # output shape (256, 7, 7）
            nn.MaxPool2d(2),                # output shape (256, 3, 3)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(256),   
        )
        self.dense1=nn.Sequential(
            nn.Linear(2304,1024),
            #nn.Dropout(0.5),
            nn.ReLU()
        )
        self.out=nn.Linear(1024,101)# fully connected layer, output 101 classes

        
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        output=self.out(x)

        return output

In [ ]:
#CNN   15MB 经济实惠，下载不用等太久
#保留dropout和两层dense的模式，可以训练到接近0.7的acc，但所需时间还是有些长150epoch左右。overfitting目前没有出现。
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (3, 224, 224)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=8,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=1,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (8, 224, 224)
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (8, 112, 112)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(8),              #使用这个normalize后，训练速度变快了！      
        )
        self.conv2 = nn.Sequential(         # input shape (8, 112, 112)
            nn.Conv2d(8, 16, 3, 1, 1),     # output shape (16, 112, 112)
            nn.MaxPool2d(2),                # output shape (16, 56, 56)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(16),        
        )
        self.conv3 = nn.Sequential(         # input shape (16, 56, 56)
            nn.Conv2d(16,32, 3, 1, 1),     # output shape (32, 56, 56)
            nn.MaxPool2d(2),                # output shape (32, 28, 28)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(32),  
        )
        self.conv4 = nn.Sequential(         # input shape (32, 28, 28)
            nn.Conv2d(32,64, 3, 1, 1),     # output shape (64, 28, 28)
            nn.MaxPool2d(2),                # output shape (64, 14, 14)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(64),
        )
        self.conv5 = nn.Sequential(         # input shape (64, 14, 14)
            nn.Conv2d(64, 128, 3, 1, 1),     # output shape (128, 14, 14)
            nn.MaxPool2d(2),                # output shape (128, 7, 7)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(128),
        )
        self.conv6 = nn.Sequential(         # input shape (128, 7, 7)
            nn.Conv2d(128, 256, 3, 1, 1),     # output shape (256, 7, 7）
            nn.MaxPool2d(2),                # output shape (256, 3, 3)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(256),   
        )
        self.dense1=nn.Sequential(
            nn.Linear(2304,1152),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense2=nn.Sequential(
            nn.Linear(1152,256),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.out=nn.Linear(256,101)# fully connected layer, output 101 classes

        
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        x=self.dense2(x)
        
        output=self.out(x)

        return output

In [ ]:
 # 1. 根据网络层的不同定义不同的初始化方式     
def weight_init(m):
    if isinstance(m, nn.Linear):
        nn.init.xavier_normal_(m.weight)
        #nn.init.constant_(m.bias, 0) bias不要全初始化为0
        nn.init.normal_(m.bias, mean=0, std=1)
    # 也可以判断是否为conv2d，使用相应的初始化方式 
    elif isinstance(m, nn.Conv2d):
        nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
     # 是否为批归一化层
    elif isinstance(m, nn.BatchNorm2d):
        nn.init.constant_(m.weight, 1)
        nn.init.constant_(m.bias, 0)
#使用这样的初始化后，模型的初始表现确实好了些

In [ ]:
#新增train_acc,valid_acc
def train_model(model,device, patience, n_epochs):
    
    # to track the training loss as the model trains
    train_losses = []
    # to track the validation loss as the model trains
    valid_losses = []
    # to track the average training loss per epoch as the model trains
    avg_train_losses = []
    # to track the training accuracy per epoch as the model trains
    train_accuracies = [] 
    # to track the average validation loss per epoch as the model trains
    avg_valid_losses = [] 
    # to track the validation accuracy per epoch as the model trains
    valid_accuracies = [] 
    # initialize the early_stopping object
    early_stopping = EarlyStopping("val_acc",patience=patience, verbose=True,delta=0)
    
    for epoch in range(1, n_epochs + 1):
 
        ###################
        # train the model #
        ###################
        model.train() # prep model for training
        train_correct=0
        for batch, (X, y) in enumerate(train_loader):
            X, y = X.to(device), y.to(device)
            # clear the gradients of all optimized variables
            optimizer.zero_grad()
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the loss
            loss = loss_func(output, y)
            # backward pass: compute gradient of the loss with respect to model parameters
            loss.backward()
            # perform a single optimization step (parameter update)
            optimizer.step()
            # record training loss
            train_losses.append(loss.item())
            # update the train_correct label
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            train_correct += pred.eq(y.view_as(pred)).sum().item() 
        ######################    
        # validate the model #
        ######################
        model.eval() # prep model for evaluation
        # to mark the correct label as the model trains
        val_correct = 0
        for batch, (X, y) in enumerate(valid_loader):
            X, y = X.to(device), y.to(device)
            # forward pass: compute predicted outputs by passing inputs to the model
            output = model(X)
            # calculate the loss
            loss = loss_func(output, y)
            # record validation loss
            valid_losses.append(loss.item())
            # update the val_correct label
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            val_correct += pred.eq(y.view_as(pred)).sum().item() 
        # print training/validation statistics 
        # calculate average loss over an epoch
        train_loss = np.average(train_losses)
        valid_loss = np.average(valid_losses)
        avg_train_losses.append(train_loss)
        avg_valid_losses.append(valid_loss)
        # calculate valid accuracy over an epoch
        train_acc=100. * train_correct / len(train_loader.dataset)
        valid_acc=100. * val_correct / len(valid_loader.dataset)
        train_accuracies.append(train_acc)
        valid_accuracies.append(valid_acc)
        
        epoch_len = len(str(n_epochs))
        
        print_msg = (f'[{epoch:>{epoch_len}}/{n_epochs:>{epoch_len}}] ' +
                     f'train_loss: {train_loss:.5f} ' +
                     f'train_accuracy: {train_correct}/{len(train_loader.dataset)} ({train_acc:.5f})% ' +
                     f'valid_loss: {valid_loss:.5f} ' +
                     f'valid_accuracy: {val_correct}/{len(valid_loader.dataset)} ({valid_acc:.5f})%')
        
        print(print_msg)
        
        # clear lists to track next epoch
        train_losses = []
        valid_losses = []
        # early_stopping needs the validation acc to check if it has incresed, 
        # and if it has, it will make a checkpoint of the current model
        early_stopping(valid_acc, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        
    # load the last checkpoint with the best model
    model.load_state_dict(torch.load('checkpoint.pt'))
 
    return  model, avg_train_losses, avg_valid_losses,train_accuracies,valid_accuracies

In [ ]:
#先用Adam进行前期的训练
lr = 0.001
# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# create a new model with these weights
model = CNN().to(device)#这个是和设备GPU/CPU相关的系数。
model.apply(weight_init)
optimizer=torch.optim.Adam(model.parameters(),lr=lr,weight_decay=1e-5)
#from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimizer, mode="min", patience=200, factor=0.01)其实我没有用这个额。
loss_func = nn.CrossEntropyLoss()  # the target label is NOT an one-hotted

In [ ]:
n_epochs=60
patience = 20
model, train_loss, valid_loss,train_acc,valid_acc = train_model(model,device, patience, n_epochs)

***Visualizing the Loss and the Early Stopping Checkpoint***

From the plot we can see that the last Early Stopping Checkpoint was saved right before the model started to overfit.

In [ ]:
# visualize the loss as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_loss)+1),train_loss, label='Training Loss')
plt.plot(range(1,len(valid_loss)+1),valid_loss,label='Validation Loss')

# find position of lowest validation loss
minposs = valid_loss.index(min(valid_loss))+1 
plt.axvline(minposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('loss')
plt.ylim(0, 0.5) # consistent scale
plt.xlim(0, len(train_loss)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('loss_plot.png', bbox_inches='tight')

In [ ]:
# visualize the accuracy as the network trained
fig = plt.figure(figsize=(10,8))
plt.plot(range(1,len(train_acc)+1),train_acc, label='Training Accuracy')
plt.plot(range(1,len(valid_acc)+1),valid_acc,label='Validation Accuracy')

# find position of lowest validation loss
maxposs = valid_acc.index(max(valid_acc))+1 
plt.axvline(maxposs, linestyle='--', color='r',label='Early Stopping Checkpoint')

plt.xlabel('epochs')
plt.ylabel('Accuracy')
plt.ylim(0, 100) # consistent scale
plt.xlim(0, len(train_acc)+1) # consistent scale
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig('Accuracy_plot.png', bbox_inches='tight')

In [ ]:
save_model = True
if (save_model):
    torch.save(model,"cnn.pkl")
    torch.save(model.state_dict(),"cnn_parameters.pkl")

In [ ]:
def test(model, device, test_loader,loss_func):
    model.eval()
    test_loss =[]
    correct = 0
    with torch.no_grad():
        for X, y in test_loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            test_loss.append( loss_func(output, y).item())
            pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
            correct += pred.eq(y.view_as(pred)).sum().item()

    test_loss = np.average(test_loss)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
test(model, device, test_loader,loss_func)

看清前期训练的表现后我再用SGD去尝试做些突破

In [ ]:
model = torch.load('cnn256.pkl').to(device)#这个是和设备GPU/CPU相关的系数。

In [ ]:
#利用SGD去突破，确实可以把acc提升些
lr = 0.001
momentum = 0.5
# Detect if we have a GPU available
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=momentum)
n_epochs=100 #100轮就差不多了，我没有能够磨上去到80%，最高也就79.28%根据512的表现，我现在感觉是可以慢慢磨上去的。
patience = 20#我觉得30真的是等太久了，后悔了额
model, train_loss, valid_loss,train_acc,valid_acc = train_model(model,device, patience, n_epochs)

In [ ]:
#再看一遍在test中的表现
test(model, device, test_loader,loss_func)

第二版CNN

In [ ]:
#CNN 66MB 版  下起来特别慢，准确率和256的差不多，maxpool在relu后面，训练超过800epoch，目前最好的成果，但耗费的时间特别长。
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (3, 224, 224)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=16,            # n_filters
                kernel_size=5,              # filter size
                stride=1,                   # filter movement/step
                padding=2,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (8, 224, 224)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (8, 112, 112)
        )
        self.conv2 = nn.Sequential(         # input shape (8, 112, 112)
            nn.Conv2d(16, 32, 5, 1, 2),     # output shape (16, 112, 112)

            nn.ReLU(),                      # activation
            nn.BatchNorm2d(32),
            nn.MaxPool2d(2),                # output shape (16, 56, 56)
        )
        self.conv3 = nn.Sequential(         # input shape (16, 56, 56)
            nn.Conv2d(32, 64, 5, 1, 2),     # output shape (32, 56, 56)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(64),
            nn.MaxPool2d(2),                # output shape (32, 28, 28)
        )
        self.conv4 = nn.Sequential(         # input shape (32, 28, 28)
            nn.Conv2d(64, 128, 5, 1, 2),     # output shape (64, 28, 28)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(128),
            nn.MaxPool2d(2),                # output shape (64, 14, 14)
        )
        self.conv5 = nn.Sequential(         # input shape (64, 14, 14)
            nn.Conv2d(128, 256, 5, 1, 2),     # output shape (128, 14, 14)
        
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(256),
            nn.MaxPool2d(2),                # output shape (128, 7, 7)
        )
        self.conv6 = nn.Sequential(         # input shape (128, 7, 7)
            nn.Conv2d(256, 512, 5, 1, 2),     # output shape (256, 7, 7）
    
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(512),
            nn.MaxPool2d(2),                # output shape (256, 3, 3)
        )
        self.dense1=nn.Sequential(
            nn.Linear(4608,2304),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense2=nn.Sequential(
            nn.Linear(2304,256),
            nn.Dropout(0.5),
            nn.ReLU()
        )
        self.out=nn.Linear(256,101)# fully connected layer, output 101 classes

        
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        x=self.dense2(x)
        
        output=self.out(x)

        return output

In [ ]:
#CNN 66MB 版  下起来特别慢，maxpooling在relu前面，符合我在stats302的编码风格，训练结果和前一版差不多（从前50epoch看来）
#去除dropout后确实会体现overfitting这样的问题。
class CNN(torch.nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Sequential(         # input shape (3, 224, 224)
            nn.Conv2d(
                in_channels=3,              # input height
                out_channels=16,            # n_filters
                kernel_size=3,              # filter size
                stride=1,                   # filter movement/step
                padding=1,                  # if want same width and length of this image after Conv2d, padding=(kernel_size-1)/2 if stride=1
            ),                              # output shape (8, 224, 224)
            nn.MaxPool2d(kernel_size=2),    # choose max value in 2x2 area, output shape (8, 112, 112)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(16),
            
        )
        
        self.conv2 = nn.Sequential(         # input shape (8, 112, 112)
            nn.Conv2d(16, 32, 3, 1, 1),     # output shape (16, 112, 112)
            nn.MaxPool2d(2),                # output shape (16, 56, 56)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(32),
            
        )
        self.conv3 = nn.Sequential(         # input shape (16, 56, 56)
            nn.Conv2d(32, 64, 3, 1, 1),     # output shape (32, 56, 56)
            nn.MaxPool2d(2),                # output shape (32, 28, 28)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(64),
            
        )
        self.conv4 = nn.Sequential(         # input shape (32, 28, 28)
            nn.Conv2d(64, 128, 3, 1, 1),     # output shape (64, 28, 28)
            nn.MaxPool2d(2),                # output shape (64, 14, 14)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(128),
            
        )
        self.conv5 = nn.Sequential(         # input shape (64, 14, 14)
            nn.Conv2d(128, 256, 3, 1, 1),     # output shape (128, 14, 14)
            nn.MaxPool2d(2),                # output shape (128, 7, 7)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(256),
            
        )
        self.conv6 = nn.Sequential(         # input shape (128, 7, 7)
            nn.Conv2d(256, 512, 3, 1, 1),     # output shape (256, 7, 7）
            nn.MaxPool2d(2),                # output shape (256, 3, 3)
            nn.ReLU(),                      # activation
            nn.BatchNorm2d(512),
   
        )
        self.dense1=nn.Sequential(
            nn.Linear(4608,2048),
            #nn.Dropout(0.5),
            nn.ReLU()
        )
        self.dense2=nn.Sequential(
            nn.Linear(2048,1024),
            #nn.Dropout(0.5),
            nn.ReLU()
        )
        self.out=nn.Linear(1024,101)# fully connected layer, output 101 classes

        
    def forward(self, x):
        x=self.conv1(x)
        x=self.conv2(x)
        x=self.conv3(x)
        x=self.conv4(x)
        x=self.conv5(x)
        x=self.conv6(x)
        x=x.view(x.size(0),-1)
        x=self.dense1(x)
        x=self.dense2(x)
        
        output=self.out(x)

        return output

处理方法和前一个CNN差不多。

In [ ]:
#这个是应对Kaggle下载困难的解决办法。
#here are the links of the pkl documents, click the link then download them, the document is a little large,so please be patient 
from IPython.display import FileLink
FileLink("cnn256.pkl")